In [5]:
#################
# path config
#################
ROOT_FOLDER = '/Users/keerat/Documents/Research'

FILE_PATTERN = '*.jpg'

OUTPUT_FILE_EXT = '.png'

### Set to True if testset you are predicting stage2 folder
is_stg2 = False

### How much extra margin we want to include when cropping the output images
margin = 0.15
#margin = 0.4   #0.4 seems to work best for my classifier

### Input folders
TRAINSET_INPUT_FOLDER = ROOT_FOLDER + '/input/train'
TESTSET_INPUT_FOLDER = ROOT_FOLDER + '/input/test_stg2' if is_stg2 else ROOT_FOLDER + '/input/test'
ADDSET_INPUT_FOLDER = ROOT_FOLDER + '/input/additional'

### Output folders
TESTSET_OUTPUT_FOLDER = ROOT_FOLDER + '/input/test_stg2_roi_{}'.format(margin) if is_stg2 else ROOT_FOLDER + '/input/test_roi_{}'.format(margin)
TRAINSET_OUTPUT_FOLDER = ROOT_FOLDER + '/input/train_roi_{}'.format(margin)
ADDSET_OUTPUT_FOLDER = ROOT_FOLDER + '/input/additional_roi_{}'.format(margin)


### Temp working folders
TRAINSET_RESIZED_FOLDER = ROOT_FOLDER + '/input/train_resized'
TESTSET_RESIZED_FOLDER = ROOT_FOLDER + '/input/test_stg2_resized' if is_stg2 else ROOT_FOLDER + '/input/test_resized'
ADDSET_RESIZED_FOLDER = ROOT_FOLDER + '/input/additional_resized'
VISUAL_RESIZED_FOLDER = ROOT_FOLDER + '/input/visual_resized'
TRAINSET_RESIZED_MASK_FOLDER = ROOT_FOLDER + '/input/train_resized_mask'

UNET_TRAIN_SPLIT_FOLDER = ROOT_FOLDER + '/input/split_unet/train_split/'
UNET_TRAINMASK_SPLIT_FOLDER = ROOT_FOLDER + '/input/split_unet/train_mask_split/'

UNET_VAL_SPLIT_FOLDER = ROOT_FOLDER + '/input/split_unet/val_split/'
UNET_VALMASK_SPLIT_FOLDER = ROOT_FOLDER + '/input/split_unet/val_mask_split/'

#################
# other parameters
#################
ClassNames = ['Type_1', 'Type_2', 'Type_3']

from sys import platform
use_symlinks = platform == "linux" or platform == "linux2" or platform == "darwin"

seed = 20170804
split_proportion = 0.8

learning_rate = 0.0001
nbr_epochs = 400
batch_size = 32

# Size could be: 64, 80, 144, 128
img_width = 128
img_height = 128
nb_channels = 3

# Augmentation
shear_range = 0.78
zoom_range = 0.4
rotation_range = 180
vflip = True
hflip = True
width_shift_range = 0.3
height_shift_range = 0.3

# preprocessing
rescale = 1. / 255.
preprocessing_function = None

# folder name
info = 'unet' \
       + '_' + str(img_height) + 'x' + str(img_width) + 'x' + str(nb_channels) \
       + '_sp' + str(split_proportion) \
       + '_sh' + str(shear_range) \
       + '_zm' + str(zoom_range) \
       + '_rt' + str(rotation_range) \
       + '_vf' + str(int(vflip)) \
       + '_hf' + str(int(hflip)) \
       + '_ws' + str(width_shift_range) \
       + '_hs' + str(height_shift_range)

In [6]:
import json
import ntpath
import os

import cv2
import numpy as np
import glob


def resize_testset(source_folder, target_folder, dsize, pattern=FILE_PATTERN):
    print('Resizing testset ...')
    if not os.path.exists(target_folder): os.makedirs(target_folder)
    total_images = glob.glob(os.path.join(source_folder, pattern))
    total = len(total_images)
    for i, source in enumerate(total_images):
        filename = ntpath.basename(source)
        target = os.path.join(target_folder, filename.replace('.jpg', '.png'))

        img = cv2.imread(source)
        img_resized = cv2.resize(img, dsize, interpolation=cv2.INTER_CUBIC)
        cv2.imwrite(target, img_resized)
        if i % 100 == 0:
            print("Resized {}/{} images".format(i, total))


def resize_addset(source_folder, target_folder, dsize, pattern=FILE_PATTERN):
    print('Resizing additional set...')
    if not os.path.exists(target_folder): os.makedirs(target_folder)
    for clazz in ClassNames:
        if clazz not in os.listdir(target_folder):
            os.makedirs(os.path.join(target_folder, clazz))

        total_images = glob.glob(os.path.join(source_folder, clazz, pattern))
        total = len(total_images)
        for i, source in enumerate(total_images):
            filename = ntpath.basename(source)
            target = os.path.join(target_folder, clazz, filename.replace('.jpg', '.png'))

            try:
                img = cv2.imread(source)
                img_resized = cv2.resize(img, dsize, interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(target, img_resized)
            except:
                print('-------------------> error in: {}'.format(source))

            if i % 20 == 0:
                print("Resized {}/{} images".format(i, total))


def resize_trainset_and_generate_masks(dsize):
    print('Resizing train set & creating masks ...')
    INPUT_FOLDER = ROOT_FOLDER + '/input'
    annotation_json_filename = INPUT_FOLDER + '/{}_bbox.json'
    for c in ClassNames:

        annotation_json = annotation_json_filename.format(c)
        print('Loading {}'.format(annotation_json))
        with open(annotation_json) as json_file:
            data = json.load(json_file)

        OUTPUT_FOLDER = os.path.join(TRAINSET_RESIZED_MASK_FOLDER, c)
        if not os.path.exists(OUTPUT_FOLDER): os.makedirs(OUTPUT_FOLDER)

        OUTPUT2_FOLDER = os.path.join(TRAINSET_RESIZED_FOLDER, c)
        if not os.path.exists(OUTPUT2_FOLDER): os.makedirs(OUTPUT2_FOLDER)

        total = len(data)
        for i, row in enumerate(data):
            if i % 10 == 0:
                print('Processing {}/{}'.format(i, total))

            input_filename = os.path.join(INPUT_FOLDER, row['filename'])
            if not os.path.exists(input_filename):
                print('Skipped input file not exist: {}'.format(input_filename))
                continue

            basename = ntpath.basename(input_filename)
            output_filename = os.path.join(OUTPUT_FOLDER, basename.replace('.jpg', '.png'))
            if os.path.exists(output_filename):
                print('Skipped output already exist: {}'.format(output_filename))
                continue

            output2_filename = os.path.join(OUTPUT2_FOLDER, basename.replace('.jpg', '.png'))
            if os.path.exists(output2_filename):
                print('Skipped output already exist: {}'.format(output2_filename))
                continue

            annotation = row['annotations'][0]
            x = int(round(annotation['x']))
            y = int(round(annotation['y']))
            w = int(round(annotation['width']))
            h = int(round(annotation['height']))
            img = cv2.imread(input_filename)

            resized = cv2.resize(img, dsize=dsize)
            cv2.imwrite(output2_filename, resized)

            mask = np.zeros_like(img)
            mask[y:y + h, x:x + w, :] = 255

            mask = cv2.resize(mask, dsize=dsize)
            cv2.imwrite(output_filename, mask)


if __name__ == '__main__':
    resize_trainset_and_generate_masks(dsize=(img_height, img_width))

    resize_testset(TESTSET_INPUT_FOLDER, TESTSET_RESIZED_FOLDER, (img_width, img_height))

    if os.path.exists(ADDSET_INPUT_FOLDER):
        resize_addset(ADDSET_INPUT_FOLDER, ADDSET_RESIZED_FOLDER, (img_width, img_height))

Resizing train set & creating masks ...
Loading /Users/keerat/Documents/Research/input/Type_1_bbox.json
Processing 0/249
Processing 10/249
Processing 20/249
Processing 30/249
Processing 40/249
Processing 50/249
Processing 60/249
Processing 70/249
Processing 80/249
Processing 90/249
Processing 100/249
Processing 110/249
Processing 120/249
Processing 130/249
Processing 140/249
Processing 150/249
Processing 160/249
Processing 170/249
Processing 180/249
Processing 190/249
Processing 200/249
Processing 210/249
Processing 220/249
Processing 230/249
Processing 240/249
Loading /Users/keerat/Documents/Research/input/Type_2_bbox.json
Processing 0/772
Processing 10/772
Processing 20/772
Processing 30/772
Processing 40/772
Processing 50/772
Processing 60/772
Processing 70/772
Processing 80/772
Processing 90/772
Processing 100/772
Processing 110/772
Processing 120/772
Processing 130/772
Processing 140/772
Processing 150/772
Processing 160/772
Processing 170/772
Processing 180/772
Processing 190/772